<a href="https://colab.research.google.com/github/sigmunjr/TEK5030_deep_learning_torch/blob/main/TEK5030_deep_learning_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEK5030 - Deep learning Lab

The goal of this exercise is to give you some hands on experiance with building and training *deep neural networks*.

To simplify setup, this execise will be run in Jupyter notebook/colab. Still some fun exercises are hard to do in a web browser. I recommende those with a proper hardware setup to also try [EX3_live_training.py](https://gitlab.com/sigmund.rolfsjord/unik4690_deep_learning_lab/-/blob/master/EX3_live_training.py) from an older lab [TEK5030 Deep learning lab (2020)](https://gitlab.com/sigmund.rolfsjord/unik4690_deep_learning_lab). In this lab you can also find code for running detection networks live.

For those interested in learning [Tensorflow](tensorflow.org), you can try exercises from [https://github.com/sigmunjr/TEK5030_deep_learning](https://github.com/sigmunjr/TEK5030_deep_learning).

The exercise will be focused around the *deep learning framework* called [PyTorch](www.pytorch.org), as this is currently the most common framework. The differences compared too other frameworks e.g. are generally quite small. The primary goal of these frameworks is to facilitate running you code on accelerators such as **GPUs**, providing **automatic differentiation** of you code and provide **tools and structure** specific for deep learning.

A large set of guides and tutorials can be found at [pytorch.org/tutorials](pytorch.org/tutorials). I highly recommend going through these tutorials if you plan on working further with deep learning.

As the dataset is only available in the newest version of *torchvision* we need to reinstall torchvision and **restart** the kernel.



In [ ]:
try:
    import torchvision
except:
    !pip install torchvision

## Enabling and testing the GPU

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with torch:
<a id='initialize_device'></a>

In [ ]:
import torch
import numpy as np

if torch.cuda.is_available():
    device = 'cuda'
    device_nr = torch.cuda.current_device()
    print(f'Found GPU device: {device_nr} of type: {torch.cuda.get_device_name(device)}')
else:
    device = 'cpu'
    print('No GPU found')

# Exercise 1: Basic PyTorch
We will start by simply introducing a few relevant concepts in PyTorch. I will reuse variables from above cells, so if you get missing variables check those cells.

## Tensors
Tensors are the primary form for data in PyTorch. Data needs to be converted to tensors to use most of the PyTorch APIs. More on this can be found [Tensors](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)

Tensors can easily be created directly from data:

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

or from Numpy:

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np *= 3
#Convert back to numpy
print(x_np.numpy())

### Running on GPU
To run computation on GPU, you need to explicitly send the data to the GPU, with *.to* or *.gpu()*.

*This is using device set in [Enabling and testing the GPU](#Enabling-and-testing-the-GPU).*

In [ ]:
x_data_gpu = x_data.to(device)

To get it back to det cpu you can use:

In [ ]:
x_data_cpu = x_data.cpu()
#Or
x_data_cpu = x_data.to('cpu')
print(f'x_data_gpu device: {x_data_gpu.device}, x_data_cpu device: {x_data_cpu.device}')

## Automatic Differentiation

An important part of deep learning and deep learning frameworks is the automatic differentiation, you can read a more extensive guide [here](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html). You can use the gradients to jointly optimize the parameters of a deep learning model. The automatic differentiation can be enabled for tensors by setting *required_grad in the initiation or setting the property afterwards. If you use premade layers or functions like [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), the required_grad will be set to **True** by default.

Pytorch builds and keeps a computational graph with all intermediate results in memory until the .backward()

In [ ]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3)

b.requires_grad = True
z = torch.matmul(x, w) + b
loss = ((z - y) ** 2).mean()
print(f'loss: {loss} b gradients: {b.grad}')

The gradients for e.g. the loss will be computed by calling *backward* on the loss tensor. You can find the gradients with respect to e.g. *b*, in *b.grad*. If b.grad gets multiple gradients, they will all be added into b.grad. So for each run you will have to manually zero out the gradients.

In [ ]:
loss.backward()
print(f'loss: {loss} b gradients: {b.grad}')

To minimize the loss you could then e.g. run:

In [ ]:
for i in range(20):  #run 20 steps of minimization
    z = torch.matmul(x, w) + b
    loss = ((z - y) ** 2).mean()
    loss.backward()
    b.data -= b.grad
    print(f'loss: {loss}, b grad norm: {b.grad.norm()}')
    b.grad.zero_()

An output of a function where some variables **require_grad**, will also "require_grad", as they may need the gradients and data to calculate the gradients of the initial variable. In other worlds a trail of gradients will be generated downstream from an initial tensor.

**OBS!** A typical error is to keep the variables that require_grad in memory to long e.g. storing them in a list.

You can stop the gradients and therby delete the whole trail of gradients, by calling .detach() on a tensor.

```python
b.detach()
```

This will also stop the propagation of gradients from .backward() from that point, as you cannot propagate the gradients.

## Datasets and dataloaders
PyTorch also have some built inn tools for working with datasets. I recommend reading through this tutorial for more information on the topic [Data tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

For these exercises i choose a dataset with different pets [Oxford-IIIT Pet](https://www.robots.ox.ac.uk/~vgg/data/pets/). This dataset is well suited because it is big enough to be interesting, but small enough to download in reasonable time. It also has both classification labels and segmentation masks.

We will use the [dataset from torchvision](https://pytorch.org/vision/stable/datasets.html) to make things simple.

### Loading and preprocessing the data
Here we load the dataset and add a *transform* function that convert the images to float tensors between 0 and 1 and resize them to a more manageable size. We also transform the  You can read more about data transformations [here](https://pytorch.org/vision/stable/transforms.html#).

In [ ]:
import torchvision.datasets as datasets
import torchvision
from torchvision.transforms import ToTensor, PILToTensor, Compose, ConvertImageDtype, Resize, Lambda, InterpolationMode

data_transforms = Compose([Resize([64, 64]), ToTensor()])
label_transforms = Compose([Resize([64, 64], InterpolationMode.NEAREST), PILToTensor()])

label_map = ['cat', 'dog']
breads_to_cat_or_dog = [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
                        0, 0, 1, 1, 1]
"""
Transforms segmentation image from PIL Image with values [0, 1, 2] (background, border, pet) to [0, 1] (background, pet) and
Transforms labels from type of breed to cat vs dog.
 """


def transform_labels(inp):
    seg_img, label = inp
    return torch.minimum(label_transforms(seg_img) - 1, torch.tensor(1)).to(torch.float32), breads_to_cat_or_dog[label]


train_data = datasets.OxfordIIITPet('.', 'trainval', target_types=('segmentation', 'category'),
                                    transform=data_transforms, download=True, target_transform=transform_labels)
test_data = datasets.OxfordIIITPet('.', 'test', target_types=('segmentation', 'category'), transform=data_transforms,
                                   download=True, target_transform=transform_labels)

#Check first output
for img, (seg_mask, label) in test_data:
    print(f'Image with shape {img.shape} values in segmentation mask: {seg_mask.unique()} label: {label_map[label]}')
    break

### Prepare data for training with dataloader
We use dataloaders to speed up fetching of data, and for convenience methods like batching and shuffling. Dataloader expect an object with implemented *\_\_len\_\_* and *\_\_get_item\_\_* methods.
Using *num_workers* will create worker threads to read and process data.


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

### Look through the data
You should always spend some time inspecting your data. Just to make sure everything is as expected...

***OBS! Torch normally processes images on the formate Channels x Height x Width, while plt.plot, OpenCV etc. uses Height x With x Channels. Therefore we need a permute to arrange the dimensions correctly before plotting.**

In [ ]:
from matplotlib import pyplot as plt

number_of_rows = 5
for i in range(number_of_rows):
    img, (seg_mask, label) = test_data[np.random.randint(0, len(test_data))]
    figure = plt.figure(figsize=(15, 15))
    #Draw img
    figure.add_subplot(1, 2, 1)
    plt.title(label_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze().permute([1, 2, 0]))
    #Draw mask
    figure.add_subplot(1, 2, 2)
    plt.title("mask")
    plt.axis("off")
    plt.imshow(seg_mask.squeeze())
    plt.show()

# Exercise 1: Simple Convolutional Neural network (CNN)

In this execrise you are building a neural network to separate between cats and dogs. If you want, you can also try separating different breeds.


### Nearest neighbors in pixel space

For fun we can check out which images are close in pixel space. This means we will find images with similar colored pixels in the same location. Often generally dark images will match with dark images and visa-versa. This makes it clear that we cannot separate cat and dogs with a shallow classification algorithm in pixel space directly.

In [ ]:
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot


def display(display_list, titles=None):
    """ Plotting images in list """
    from matplotlib import cm
    plt.figure(figsize=(15, 15))

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        if titles != None:
            plt.title(titles[i])
        plt.imshow(
            np.squeeze(display_list[i]).transpose((1, 2, 0)),
            vmin=0,
            vmax=1
        )
        plt.axis('off')
    plt.show()


# Pick the first 400 samples and take only the images of each sample
nn_loader = DataLoader(train_data, batch_size=400, shuffle=True)

# Extract the images to a numpy array of the 400 images of shape 200x3x128x128
X = next(iter(nn_loader))[0].numpy()
print(X.shape)

nbrs = NearestNeighbors(n_neighbors=3)
# Add images reshaped to vectors to nearest-neighbors tree
nbrs.fit(X.reshape([X.shape[0], -1]))

# Loop through 10 test images and display 3 nearest neighbors
for i, (image, (mask, label)) in enumerate(test_data):
    if i >= 10:
        break
    distances, indices = nbrs.kneighbors(image.numpy().reshape((1, -1)))
    show_img = display([
        image.numpy(),
        X[indices[0, 0]],
        X[indices[0, 1]],
        X[indices[0, 2]]
    ], ['Image'] + list(distances[0]))

## Fully-convolutional network
It's a good idea to implement the network as a fully-convolutional neural network. This means that the network will work for different input sizes.

A fully-convolutional network use no fully-connected layers (matrix multiplications). They often consist of mostly convolutional layers, but also include other operations that does not require fixed input size like: add, concatenations, max (relu), batch normalization etc.

## Torch Module
I recommend using the Object-Oriented style of Torch Module, as it is the most flexible and are almost identical in both Keras/Tensorflow and PyTorch. In this approach you make subclass of [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=module#torch.nn.Module) and implement the **\_\_init\_\_** and **forward** method. In **\_\_init\_\_** you need to initialize everything that has state in your network, while **call** method actually run the network from output to input.

You can use [torch.nn.Convd2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), for the convolutions.

I also recommend using some pooling operations like [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) or [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html), or strides in the convolutions to get a larger field of view.

You can check out this tutorial [Build Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html), for pointer on how to implement a module.

**TODO:**
Fill out the *\_\_init\_\_* and *call* method with appropriate layers. Remember to use activation functions like [*ReLU*](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) between your convolutional layers, but do not apply an activation function after your final layer.

Try to use only a few layers 4-6 convolutional layers, to save computation and avoid problems with vanishing gradients. With many layers you would need to either be very careful with initialization and learning rate or use normalization layers like BatchNormalization.

The output of the *forward* function should have shape \[*batch_size*, *num_classes*\].

**If the "output_features" parameter is *True* you should output the result before the last convolutional layer.**

*[torch.mean](https://pytorch.org/docs/stable/generated/torch.mean.html#torch.mean) is used to average over the spatial dimension to get only one output vector for the whole image.*

In [ ]:
import time

import cv2
import numpy as np
import torch
from torch.nn import Conv2d, Linear, Dropout, ReLU, AvgPool2d, MaxPool2d
from torch.nn.functional import relu, elu


class SimpleNet(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleNet, self).__init__()
        self.num_classes = num_classes
        self.conv1 = Conv2d(3, 16, (3, 3))
        # TODO: Initialize the layers of your network
        # You can find different layers in torch.nn (https://pytorch.org/docs/stable/nn.html#convolution-layers)

    def forward(self, x, output_features=False):
        # TODO: Run the image through your network
        x = self.conv1(x)
        # Your input should be a [Batch_size x 3 x 32 x 32] sized tensor
        # Your output should be a [Batch_size x num_classes] sized matrix
        # Output features of your second last layer
        if output_features: return x
        # Return the result of your network
        return torch.mean(x, dim=(2, 3))


# Initializing the model
NUM_CLASSES = 5
BATCH_SIZE = 8
model = SimpleNet(NUM_CLASSES)
# Running the model with 8 random 128x128x3 images
model_output = model(torch.from_numpy(np.random.random((BATCH_SIZE, 3, 128, 128)).astype(np.float32)))
print('Model output:', model_output)
print('Model output shape:', model_output.shape)

assert model_output.shape == (BATCH_SIZE, NUM_CLASSES), "Incorrect output size from call"

### Parameters
If your using the predefined modules from *torch.nn*, the parameters of the layers are automatically added to the modules parameters. The parameters are just tensors, but the register in a hieraki, so nested modules easiely can access all parameters.

If you want to use raw tensors as parameters for your model, you need to wrap your tensors in *torch.nn.parameter.Parameter*

```python
class MyModule(nn.Module):
  def __init__(self):
    super(MyModule, self).__init__()
    self.b = torch.nn.parameter.Parameter(torch.tensor(1))
```

You can iterate over parameters in a module as following:

In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

### Training a Torch model
As we saw in [Automatic Differentiation](#Automatic-Differentiation) we can get the gradients with respect to a tensor after a backward. The tensor can also be updated with that gradient. Still this can be quite a hassel with many parameters/tensors.

PyTorch provide a simplified interface for this, with [torch.optim](https://pytorch.org/docs/stable/optim.html). An optimizer is initialized with the parameters or tensors it is supposed to optimize. With a torch.nn.Module this is simply done with e.g.:

```python
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
```

Calling *optimizer.step()* will then update the tensor based on the gradient information. Calling *optimizer.zero_grad()* will zero out all the gradients ahead of a new step. A training step could then be done as following:

```python
optimizer.zero_grad()
ouput = model(x)
loss = calculate_loss(x, target)
loss.backward()
optimizer.step()
```

In [torch.optim](https://pytorch.org/docs/stable/optim.html#algorithms) you can find many implementations of different optimization algorithms. They all subtrackt some scaled version of the gradients from the parameters, but use various information to decide the step-size/scale. For simple tests i prefere to use [Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam) as it is more robust to the input parameters than many of the other methods (you will need less parameter tuneing).

#### **Tips for training**
Start by training your network with just a small dataset, e.g. one batch. 

To check you have properly initialized your weights and scaled your inputs you can check your initial loss. Your initial loss should be roughly equal to *-ln(1/N)*, where N is the number of classes.

When training on a small version of your dataset your accuracy should steadily rise to 1. If it does not, you could try to adjust your learning rate. Your learning rate should generally be as high as possible, while still not increase your loss at the begining of your training.

Adam is often a good optimizer for quick tests, as it requires comparably less *tuning* of the learning rate.

You can quite easily calculate your own loss function, but for convinience you can use [torch.nn.functional.cross_entropy](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy).

**TODO**:
Train the network and validate that the loss goes down. The dataset are small so we cannot expect very good accuracy, additionally colab provide less resources than previously. Therefore **do not** expect a very good result, just better than random. While you could get a decent result with a small model, I recommend to move on after your model just get a loss sligtly below ln(num_classes)=0.693.

Remember to create your dataloaders by running the cells in [Datasets and dataloader](#Datasets-and-dataloader)

In [5]:
NUM_EPOCHS = 3
model = SimpleNet(num_classes=2).to(device)

#TODO: Create loss function "calculate_loss" and optmizer

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    tic = time.time()

    for i, (img, (seg_mask, label)) in enumerate(train_dataloader):

        # TODO: Run model, calculate loss and optimize parameters

        running_loss += loss
        if i % 10 == 9:  # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f, time: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10, time.time() - tic))
            running_loss = 0.0
            tic = time.time()
    loss_accum = 0
    acc_accum = 0
    cnt = 0
    for inputs_test, (seg_mask, labels_test) in test_dataloader:
        inputs_test, labels_test = inputs_test.to(device), labels_test.to(device)
        with torch.no_grad():
            test_output = model(inputs_test)
            test_loss = calculate_loss(test_output, labels_test)
        test_acc = (test_output.argmax(1) == labels_test).double().mean()
        loss_accum += test_loss.detach()
        acc_accum += test_acc.detach()
        cnt += 1
    print(f'test acc: {acc_accum/cnt} test_loss: {loss_accum/cnt}')

NameError: name 'SimpleNet' is not defined

### Visualizing nearest neighbor in feature space

The neural network transforms the input so that cats and dogs can be separated. Generally ths will make cat be close to cats and dogs be close to dogs. We can often also see that close object share some characteristics.

In [ ]:
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot

import numpy as np

# Pick the first 400 samples and take only the images of each sample
nn_loader = DataLoader(train_data, batch_size=400, shuffle=True)

# Extract the images to a numpy array of the 400 images of shape 200x3x128x128
images_np = next(iter(nn_loader))[0].numpy()

# Extract the images to a numpy array of the 200 images of shape 200x128x128x3
X = model(torch.tensor(images_np).to(device), output_features=True).detach().cpu().numpy()

nbrs = NearestNeighbors(n_neighbors=3)
# Add images reshaped to vectors to nearest-neighbors tree
print('XSHAPE', X.shape)
nbrs.fit(X.reshape([X.shape[0], -1]))

# Loop through 10 test images and display 3 nearest neighbors
cnt = 0
for image, (mask, label) in test_data:
    if cnt == 10:
        break
    cnt += 1
    distances, indices = nbrs.kneighbors(
        model(image[None].to(device), output_features=True).detach().cpu().numpy().reshape((1, -1)))
    show_img = display([
        image.numpy(),
        images_np[indices[0, 0]],
        images_np[indices[0, 1]],
        images_np[indices[0, 2]]
    ], ['Image'] + list(distances[0]))

### Visualizing [GRAD-CAM](http://openaccess.thecvf.com/content_ICCV_2017/papers/Selvaraju_Grad-CAM_Visual_Explanations_ICCV_2017_paper.pdf)

There are many techniques for visualizing what is going on inside a neural network.

The technique use the gradients of the output label with respect to the image in combination with the values of the activation map. With this information we can plot what regions of the input that are pointing toward a given label.

We would e.g. expect that a cat would light up if we used the cat label for as input to Grad-CAM.

With this technique we can check if the network actually use relvant information in the classification process and are not overfitted.

Since the output is normalized, it will always highlight some regions. If the surroundings is highlighted it may simply mean that it found nothing "doglike" in the image.

We use the visualization tool [Captum](https://captum.ai/) for visualization.

In [ ]:
#Install the visualization tool
!pip install captum

In [ ]:
from captum import attr
from captum.attr import GuidedGradCam

#Create gradcam for model
#TODO: input model and models last layer necessarily not conv5
gradcam = GuidedGradCam(model, model.conv5)

cnt = 0
label_map = ['CAT', 'DOG']

for idx in np.random.choice(len(test_data), 10):
    image, (mask, label) = test_data[idx]
    if cnt == 10:
        break
    cnt += 1
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    ax[0].imshow(image.permute(1, 2, 0).numpy())
    ax[1].imshow(image.permute(1, 2, 0).numpy())
    out = model(image[None].to(device)).softmax(1).detach()
    image.requires_grad = True
    attr_cat = gradcam.attribute(image[None].to(device), 0, interpolate_mode='bicubic').mean((0, 1)).detach().cpu()
    attr_dog = gradcam.attribute(image[None].to(device), 1, interpolate_mode='bicubic').mean((0, 1)).detach().cpu()
    print(attr_cat.shape)
    attr_cat -= attr_cat.min()
    attr_cat /= attr_cat.max() + 1e-8
    attr_dog -= attr_dog.min()
    attr_dog /= attr_dog.max() + 1e-8

    ax[0].set_title('CAT')
    ax[1].set_title('DOG')

    ax[0].matshow(attr_cat, cmap='jet', alpha=0.8 * attr_cat)
    ax[1].matshow(attr_dog, cmap='jet', alpha=0.8 * attr_dog)

    plt.show()
    print(f'{label_map[out[0].argmax()]}: {out.max()}')


### Finetuning a network
You can quite easily download and run a pretrained network. Using a network that is trained on a similar, but larger dataset will often improve your result. Using a network pretrained on the ImageNet dataset will definetly work, as the dataset already includes different types of dogs and cats.

You can e.g. load a pretrained network like this:

    from torchvision import models
    weights = models.ResNet18_Weights.DEFAULT
    base_model = models.resnet18(weights=weights)

We then want to swap out the top classification layer, as we want a different number of classes. So we have to make this layer for our self. You can check out the code for [resnet18
here](https://github.com/pytorch/vision/blob/31a4ef9f815a86a924d0faa7709e091b5118f00d/torchvision/models/resnet.py#L266). By reading the code, you can se that the final layer can be swapped out by doing,

    base_model.fc = ...

To get maximal performance from the weights, we should also use the same preprocessing as used when training the pretrained weights. This can be done with:

    transforms = weights.transforms()
    img_preprocessed = transforms(images)

Now the network can be used similar to a Conv2d layer. If you have a small dataset or few computational resourses, you can freeze the network parameters in the pretrained network like this:

    for param in net.parameters():
        param.requires_grad = False
    
Since the network is trained on ImageNet with a 1000 classes, your loaded network will have 1000 outputs. There are two common options to fix this.

1. The simplest way is to override the networks fc layer e.g. like this:
```
num_ftrs  = self.base_model.fc.in_features
net.fc = Linear(num_ftrs, num_classes)
```

2. Another more flexible, but more complicated method is to just use the pretrained modules submodules directly in your forward function.

```python
def forward(self, x):
  x = self.net.conv1(x)
  x = self.net.bn1(x)
  ...
```

Here you have to look into how the base networks original forward function works.

**TODO:**
Use a pretrained network as a start of your model and create only the last layers. Train only the layers you have created.

In [ ]:
import time
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import torch.nn as nn
import torch
import os


class FinetuneNet(nn.Module):
    def __init__(self, num_classes=10):
        super(FinetuneNet, self).__init__()
        self.num_classes = num_classes
        # TODO: Initialize the layers of your network

    def forward(self, x, output_features=False):
        # TODO: Run the image through your network
        # Your input should be a [Batch_size x 3 x 32 x 32] sized tensor
        # Your output should be a [Batch_size x num_classes] sized matrix
        if output_features: return x
        # Return the result of your network
        return x


# Initializing the model
NUM_CLASSES = 5
BATCH_SIZE = 8
model_fine = FinetuneNet(NUM_CLASSES).to(device)
# Running the model with 8 random 128x128x3 images
model_output = model_fine(torch.rand((BATCH_SIZE, 3, 128, 128)).to(device))
print('Model output:', model_output)
print('Model output shape:', model_output.shape)

assert model_output.shape == (BATCH_SIZE, NUM_CLASSES), "Incorrect output size from call"

**TODO:**

Train your new network.

This should give a lot better performance, at least 0.9 in test accuracy!

You can re-run the visualization code, with this model. This should give a better clustering of dog and cats, and even cluster different breeds.

**TODO:** Initialize and finetune your model

In [ ]:
#Create loader with batch size that fits in memory
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=6)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=1)
num_classes = 2

Alternatively run this to test with the cifar dataset instead:

In [ ]:
from torchvision import datasets


def add_dummy_segmentation():
    def transform(labels):
        return 0, labels

    return transform


data_transforms = Compose([ToTensor()])
cifar_trainset = datasets.CIFAR10(root='.', train=True, download=True, transform=data_transforms,
                                  target_transform=add_dummy_segmentation())
cifar_testset = datasets.CIFAR10(root='.', train=False, download=True, transform=data_transforms,
                                 target_transform=add_dummy_segmentation())
num_classes = 10

train_dataloader = DataLoader(cifar_testset, batch_size=32, shuffle=True, num_workers=6)
test_dataloader = DataLoader(cifar_testset, batch_size=128, shuffle=False, num_workers=1)

In [ ]:
NUM_EPOCHS = 3
# TODO: Initialize and finetune your model


#TODO: Create optimizer and loss function

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    tic = time.time()

    for i, (img, (seg_mask, label)) in enumerate(train_dataloader):
        # TODO: Run model, calculate loss and optimize parameters

        running_loss += loss
        if i % 10 == 9:  # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f, time: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10, time.time() - tic))
            running_loss = 0.0
            tic = time.time()
    loss_accum = 0
    acc_accum = 0
    cnt = 0
    for inputs_test, (seg_mask, labels_test) in test_dataloader:
        inputs_test, labels_test = inputs_test.to(device), labels_test.to(device)
        with torch.no_grad():
            test_output = model_fine(inputs_test)
            test_loss = calculate_loss(test_output, labels_test)
        test_acc = (test_output.argmax(1) == labels_test).double().mean()
        loss_accum += test_loss.detach()
        acc_accum += test_acc.detach()
        cnt += 1
    print('TEST [%d, %5d] loss: %.3f, acc: %.3f' %
          (epoch + 1, i + 1, loss_accum / cnt, acc_accum / cnt))

# Exercise 3: Segmentation
We use the same dataset as in Exercise 1, but now instead of classifying dogs vs. cats, we try to output a segmentation mask. In other words simply classify each pixel, for this dataset simply into animal and not animal.

First we load the dataset, just as last time.

## Creating a segmentation network

A segmentation network is very similar to a classification network, except that your output need to be in the same spatial dimentions as your input.

### Simple segmentation network
The simplest way of doing this is to run a fully-convolutional network (only convolutionas and …Batch-size\*C\*N\*M, then N and M should be larger than 1 (probably N, M > 5).

You will probably get best results by using a pretrained network, but then you may need to use the second method of rewriting the *forward* function.

I use [torch.nn.functional.interpolate](https://pytorch.org/docs/stable/generated/torch.nn.functional.interpolate.html) to resize the output of the network. This implements gradients, so we can run backpropagation through the function.

## Loss function
A typical approach is to let your network output a classification for each pixel. So instead of your network outputting a tensor of shape (N, K), you output a tensor of shape (N, K, H, W). Here N is the number of input images, K is the number of classes, while H and W are height and width. 

Each output along the K dimension should then represent the loglikelihood of a given class, for that pixel.

Luckily both [torch.nn.functional.cross_entropy](https://pytorch.org/docs/stable/generated/torch.nn.functional.binary_cross_entropy.html#torch.nn.functional.binary_cross_entropy)and [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) support arbitrary tensor size.

In [ ]:
import time

import numpy as np
import torch
from torch.nn import Conv2d, Linear, Dropout
from torch.nn.functional import interpolate
from torch.nn.functional import relu


class SimpleSegNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(SimpleSegNet, self).__init__()
        self.num_classes = num_classes
        # TODO: Initialize the layers of your network
        # You can find different layers in torch.nn

    def forward(self, x, output_features=False):
        in_size = x.shape
        # TODO: Run the image through your network
        # Your input should be a [Batch_size x 3 x 32 x 32] sized tensor
        # Your output should be a [Batch_size x num_classes] sized matrix

        # Output features of your second last layer
        if output_features: return x

        # Return the result of your network
        return interpolate(x, in_size[2:4], mode='bilinear')

    @staticmethod
    def compute_loss(pred, label):
        #TODO: Implement loss function
        return torch.tensor(0.0)

## Train your network
Try to train your network both with a premade loss and your own implementation of a loss function.

I suggest something as simple ass L2 loss (mean squared error).


In [ ]:
NUM_EPOCHS = 3
seg_model = SimpleSegNet(num_classes=2).to(device)
model_fine = None
model = None
running_loss = 0.0

#TODO: Create Optimizer

calculate_loss = torch.nn.CrossEntropyLoss()
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    tic = time.time()

    for i, (img, (seg_mask, label)) in enumerate(train_dataloader):
        # TODO: Run model, calculate loss and optimize parameters

        running_loss += loss.detach()
        if i % 10 == 9:  # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f, time: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10, time.time() - tic))
            running_loss = 0.0
            tic = time.time()
    test_losses = []
    test_accs = []
    for inputs_test, (seg_mask, labels_test) in test_dataloader:
        inputs_test, seg_mask = inputs_test.to(device), seg_mask.to(device)
        with torch.no_grad():
            test_output = seg_model(inputs_test)
            test_losses.append(calculate_loss(test_output, seg_mask[:, 0].to(torch.int64)).cpu().numpy())
            test_accs.append((test_output.argmax(1) == seg_mask).double().mean().cpu().numpy())
    print('TEST [%d, %5d] loss: %.3f, acc: %.3f' %
          (epoch + 1, i + 1, np.mean(test_losses), np.mean(test_accs)))


## Visualizing your result

**TODO**: Inside the loop iterating through the test data, run your model and plot the result.

In [ ]:
def display(display_list, titles=None, cmaps={}):
    """ Plotting images in list """
    from matplotlib import cm
    from matplotlib import pyplot as plt
    plt.figure(figsize=(15, 15))

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        if titles != None:
            plt.title(titles[i])
        plt.imshow(
            display_list[i],
            vmin=0,
            vmax=1,
            cmap=cmaps[i] if i in cmaps else None
        )
        plt.axis('off')
    plt.show()


cnt = 0
for image, (mask, label) in test_data:
    if cnt == 4:
        break
    cnt += 1
    # TODO: Get the output from your network
    out = torch.nn.functional.softmax(seg_model(image[None].to(device)), 1).cpu().detach().numpy()[0, label, :, :]

    #TODO: plot the output
    display([image.permute(1, 2, 0).numpy(), mask.numpy().squeeze(), out.squeeze()], cmaps={2: 'jet'})

## Extra challenge

Using the streigh forward fully-convolutional approach is simple, but you do lose much of the spatial information, makeing the network less precise. A better approach is to us a [U-Net](https://arxiv.org/pdf/1505.04597.pdf) type architecture. Here you make skip-connection, either by simple addition or by concatinating the outputs. This can be done by only changing the *call* method in the network. Then the output can be resized multiple times with tf.image.resize and the skip-connection can be done with [torch.cat](https://pytorch.org/docs/stable/generated/torch.cat.html) or simply +. It is not necessary to use max-pooling or up-convolution/Conv2DTransposed.

![u-net](https://media.springernature.com/m685/springer-static/image/art%3A10.1038%2Fs41598-019-53797-9/MediaObjects/41598_2019_53797_Fig1_HTML.png)